# Test DuckDB access of parquet files in S3 using Sagemaker

going to query some parquet files in S3 using DuckDB and see how it performs

Need to manually install some libraries on Sagemaker:

In [ ]:
%pip install awswrangler
%pip install duckdb

In [ ]:
import duckdb
import awswrangler as wr
import boto3

real path and secrets all scrubbed 

In [ ]:
my_parquet_list = wr.s3.list_objects('s3://real_path_removed/df1/*.parquet')


Sagemaker default permissions are not able to read files written by my other account so I'm forcing key and tokens. 

In [ ]:
region_name="eu-west-1"
AWS_ACCESS_KEY_ID="xxx"
AWS_SECRET_ACCESS_KEY="xxx"
AWS_SESSION_TOKEN="xxx"

In [ ]:
con = duckdb.connect()
con.execute('INSTALL httpfs;')

set_settings = con.execute(f"""
load httpfs;
SET s3_region='eu-west-1';
SET s3_access_key_id='{AWS_ACCESS_KEY_ID}';
SET s3_secret_access_key='{AWS_SECRET_ACCESS_KEY}';
SET s3_session_token='{AWS_SESSION_TOKEN}';            
""")

The test is to read 101 files which is approx 165MB and 11.5m records. 

In [ ]:
%%timeit
df = con.execute(f"select sum(value) from parquet_scan({my_parquet_list[0:100]}) ;").df()